## 前処理用のコード

- jukyoは使わない
- 地積の公募値と実測値が異なるかを示すフラグを入れる
- 用途が二つある場合があるので、建ぺい率の大きい方を採用し、yoto, kempei, yosekiの各列にする。

In [54]:
import numpy as np
import pandas as pd
import math

In [55]:
train_genba = pd.read_csv("data/train_genba.tsv",sep='\t')

In [56]:
genba = preprocess(train_genba)

In [57]:
genba.to_csv('data/processed_train_genba.csv')

In [52]:
def preprocess(genba):
    df = genba.copy()
    
    # j# TODO: jukyoを削除
    # df = df.drop('jukyo',axis=1)
    
    # 地積の公募値と実測値の差を特徴量に追加
    df['chiseki_diff'] = df['chiseki_kb_hb']-df['chiseki_js_hb']
    
    # 二つの用途から、一つの用途、建ぺい率、容積率を返却する
    df[['yoto','kempei','yoseki']] = df.apply(select_yoto, axis=1)
    
    # 上水種別を数値化
    df['josui'] = df.apply(lambda x: 1 if x['josui']=='公営' else 0, axis=1)
    
    # 下水種別を数値化
    df['gesui'] = df.apply(lambda x: 1 if x['gesui']=='公共下水' else 0, axis=1)
    
    # ガス種別を数値化
    df['gas'] = df.apply(lambda x: 2 if x['gas'] =='都市ガス' else 1 if x['gas'] == '集中プロパン' else 0, axis=1)
    
    # 雨水（usui）をダミー変数化
    # TODO: usuiを削除
    df = pd.merge(df, pd.get_dummies(df[['usui']],drop_first = False), left_index = True, right_index = True)
    
    # 欠損値処理
    # 建て売り数 0で補完（trainデータのID:726は、一件だけの販売なので、他の建て売り数は０で正しい
    df = df.fillna({'tateuri_su': 0, 'tochiuri_su':0 , 'joken_su':0 })
    
    # 同現場での階数ごとの販売数は、gotoデータとつきあわせると正確に出そうだが、とりあえず０で補完する。
    df = df.fillna({'hy1f_date_su':0, 'hy2f_date_su':0, 'hy3f_date_su':0})
    
    # 道路
    # 扱いが難しい。road1〜road4まであるが、road1だけのものもあれば、road1〜4まで４つあるものまで。
    # とりあえず、road1だけを採用し、接する道路数を表す項目を追加することにする。
    # Future TODO
    # road2〜road4のデータの採用のしかた
    
     # road1
    # road1_hkとroad1_sbをダミー変数化する。
    # road2〜road4の有無で、道路数を表すroad_numを作る
    # TODO: road2〜road4を削除
    df = pd.merge(df, pd.get_dummies(df[['road1_hk','road1_sb']],drop_first = False), left_index = True, right_index = True)
    df['road_num']=df.apply(lambda x: 4 if x['road4_fi']!=0 else 3 if x['road3_fi']!=0 else 2 if x['road2_fi']!=0 else 1, axis=1)
    
    #kaoku_umとyheki_umuを0と1で置き換え
    df['kaoku_um']=df.apply(lambda x: 0 if x['kaoku_um']=="（無）" else 1, axis=1)
    df['yheki_umu']=df.apply(lambda x: 0 if x['yheki_umu']=="（無）" else 1, axis=1)
    
    #yheki_yohiを0と1で置き換え
    df['yheki_yohi']=df.apply(lambda x: 0 if x['yheki_yohi']=="（不要）" else 1, axis=1)
    
    #kborjsを0と1で置き換え。"公募"に変換ミスがあって、３通りあるため、「実測」で値を決める
    df['kborjs']=df.apply(lambda x: 0 if x['kborjs']=='実測' else 1, axis=1)
    
    # 引渡の状態 hw_status
    # 善し悪しを表しそうなので数値化したいが、とりあえずはダミー変数化する。
    # TODO: hw_statusを削除
    df = pd.merge(df, pd.get_dummies(df[['hw_status']],drop_first = False), left_index = True, right_index = True)
    
    #都市計画区域別 toshikuiki1とtoshikuiki２　toshikuiki2があればそちらを採用する。
    #一度toshikuiki2がカラの場合にtoshikuiki(新設)に特殊な値を入れて比較しないとうまくいかない。他に方法がありそうだが
    # TODO: toshikuiki1,toshikuiki2,toshikuikiを削除
    df['toshikuiki']=df['toshikuiki2'].fillna('empty')
    df['toshikuiki'] = df.apply(lambda x: x['toshikuiki1'] if x['toshikuiki']=="empty" else x['toshikuiki2'], axis=1)
    df = pd.merge(df, pd.get_dummies(df[['toshikuiki']],drop_first = False), left_index = True, right_index = True)
    
    # 高度地区(kodochiku) ダミー変数化する
    df = pd.merge(df, pd.get_dummies(df[['kodochiku']],drop_first = False), left_index = True, right_index = True)
    
    # chikukeikaku, keikakuroad, kaihatsukyoka, t53kyoka, hokakyokaを0,1に変換
    df['chikukeikaku']=df.apply(lambda x: 0 if x['chikukeikaku']=="（無）" else 1, axis=1)
    df['keikakuroad']=df.apply(lambda x: 0 if x['keikakuroad']=="（無）" else 1, axis=1)
    df['kaihatsukyoka']=df.apply(lambda x: 0 if x['kaihatsukyoka']=="（不要）" else 1, axis=1)
    df['t53kyoka']=df.apply(lambda x: 0 if x['t53kyoka']=="（不要）" else 1, axis=1)
    df['hokakyoka']=df.apply(lambda x: 0 if x['hokakyoka']=="（不要）" else 1, axis=1)
    
    # 防火地区をダミー変数化
    df = pd.merge(df, pd.get_dummies(df[['bokachiiki']],drop_first = False), left_index = True, right_index = True)
    
    # 最低敷地面積(minmenseki) 欠損がある。ゼロで補完する
    df['minmenseki'] = df['minmenseki'].fillna(0)
    
    # kinshijikoを0,1に変換
    df['kinshijiko']=df.apply(lambda x: 0 if x['kinshijiko']=="（無）" else 1, axis=1)
    
    # 路線価格(rosenka_hb)  ... 欠損が多いが、価格に相関ありそう。下手に補完するとまずいかもしれないので、この項目を落とす
    # Future TODO : まともな補完手段を考える
    df = df.drop('rosenka_hb',axis=1)

    
    return df

### 様々な関数

#### data_definition.txtによると、用途２がある場合には用途２を使う、とあった。
#### 当初は、建ぺい率の大きい方のyotoを採用し、yoto, kempei, yosekiからなるSeriesを返却する　とした。

In [25]:
def select_yoto(x):
    if ( x['yoto2']==np.nan):
        yoto =x['yoto1']
        kenpei =x['kempek1']
        yoseki =x['yoseki1']
    else:
        yoto =x['yoto2']
        kempei = x['kempei2']
        yoseki =x['yoseki2']    
    """"
    if ( x['yoto2']==np.nan):
        yoto =x['yoto1']
        kenpei =x['kempek1']
        yoseki =x['yoseki1']
    elif ( x['kempei1']<x['kempei2']):
        yoto =x['yoto1']
        kempei = x['kempei1']
        yoseki =x['yoseki1']
    else:
        yoto = x['yoto']=x['yoto1']
        kempei = x['kempei']=x['kempei1']
        yoseki =x['yoseki1']
    """
    return pd.Series([yoto, kempei, yoseki])